In [1]:
%%time
import time
import json
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
from tinkoff.invest import Client, InstrumentStatus, InstrumentIdType, CandleInterval, HistoricCandle
from datetime import datetime, timedelta

def mmscaler(df_col, mi=-1, ma=-1):
    if mi==-1: mi = df_col.min() 
    if ma==-1: ma = df_col.max()
    df_col = (df_col - mi) / (ma - mi)
    return df_col

def create_df(candles: [HistoricCandle]):
    df = pd.DataFrame([{
        'time': c.time, 'volume': c.volume,
        'open': c.open.units + c.open.nano / 1e9,
        'close': c.close.units + c.close.nano / 1e9,
        'high': c.high.units + c.high.nano / 1e9,
        'low': c.low.units + c.low.nano / 1e9} for c in candles])
    return df

try:
    DF
    print('DF is already defined')
except:    
    creds = json.load(open('tcreds.json', encoding='utf-8')) # подставить свое
    trtoken = creds['trtoken']
    
    with Client(trtoken) as client:
        attempt = 0
        while True:
            try:
                DF = pd.DataFrame()
                for i in range(7, 2100, 7):
                    r = client.market_data.get_candles(
                        figi="BBG004730ZJ9", # VTBR
                        from_=datetime.utcnow() - timedelta(days=i),
                        to=datetime.utcnow()-timedelta(days=i-7),
                        interval=CandleInterval.CANDLE_INTERVAL_HOUR
                    )
                    df = create_df(r.candles)
                    DF = pd.concat([df, DF], axis=0, ignore_index=True)
                break
            except:
                print('attempt №', attempt:=attempt+1)
                time.sleep(attempt if attempt<60 else 60)
    
    DF['dayofyear'] = DF.time.dt.dayofyear
    DF['weekday'] = DF.time.dt.weekday
    DF['utchour'] = DF.time.dt.hour
    DF.drop(['time'], axis=1, inplace=True)
    
    y = time.localtime().tm_year
    mon = time.localtime().tm_mon if time.localtime().tm_mon > 9 else f'0{time.localtime().tm_mon}'
    d = time.localtime().tm_mday if time.localtime().tm_mday > 9 else f'0{time.localtime().tm_mday}'
    h = time.localtime().tm_hour if time.localtime().tm_hour > 9 else f'0{time.localtime().tm_hour}'
    mins = time.localtime().tm_min if time.localtime().tm_min > 9 else f'0{time.localtime().tm_min}'
    sec = time.localtime().tm_sec if time.localtime().tm_sec > 9 else f'0{time.localtime().tm_sec}'

    DF.to_json(fr'C:\SEG\myproj\trobot\VTBR_{y}_{mon}_{d}_{h}_{mins}_{sec}.json') # подставить свое, либо закомментировать, если нет необходимости
    print('DF is saved')

x_train = list()
bullops = list()
bearops = list()

for i in range(DF.shape[0]-235):
    dfs = DF[i:i+236].copy()
    close = dfs.tail(1)['close'].values[0]
    max_ = DF[i+237:i+237+17]['high'].max()
    min_ = DF[i+237:i+237+17]['high'].min()
    bull = (max_ - close) / close
    bear = (min_ - close) / close
    
    dfs.volume = mmscaler(dfs.volume)
    dfs.open = mmscaler(dfs.open)
    dfs.close = mmscaler(dfs.close)
    dfs.high = mmscaler(dfs.high)
    dfs.low = mmscaler(dfs.low)
    dfs.dayofyear = mmscaler(dfs.dayofyear, ma=DF.dayofyear.max(), mi=DF.dayofyear.min())
    dfs.weekday = mmscaler(dfs.weekday, ma=DF.weekday.max(), mi=DF.weekday.min())
    dfs.utchour = mmscaler(dfs.utchour, ma=DF.utchour.max(), mi=DF.utchour.min())
    
    x_train.append(np.array(dfs))
    if bull >= 0.03:
        bullops.append(1)
    else:
        bullops.append(0)
    if bear <= -0.03:
        bearops.append(1)
    else:
        bearops.append(0)

x_train = np.array(x_train)
bullops = np.array(bullops)
bearops = np.array(bearops)

print(DF.tail())
print(DF.shape)

DF is saved
       volume      open     close      high       low  dayofyear  weekday  \
14488     494  0.016520  0.016525  0.016525  0.016385         71        6   
14489     631  0.016390  0.016530  0.016530  0.016385         71        6   
14490     978  0.016395  0.016390  0.016530  0.016385         71        6   
14491    1918  0.016520  0.016525  0.016530  0.016380         71        6   
14492     827  0.016525  0.016525  0.016530  0.016390         71        6   

       utchour  
14488       11  
14489       12  
14490       13  
14491       14  
14492       15  
(14493, 8)
CPU times: total: 43.5 s
Wall time: 52.8 s


In [2]:
%%time
# Обучениe быка
y_train_cat_bull = keras.utils.to_categorical(bullops, 2)

model_bull = keras.Sequential([
    Flatten(input_shape=(236, 8, 1)),
   # Dense(944, activation='relu'),
    # Dense(944, activation='relu'),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')])

model_bull.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model_bull.fit(x_train, y_train_cat_bull, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
179/179 [==============================] - 1s 5ms/step - loss: 0.4478 - accuracy: 0.8435 - val_loss: 0.5178 - val_accuracy: 0.8412
Epoch 2/5
179/179 [==============================] - 1s 4ms/step - loss: 0.4007 - accuracy: 0.8510 - val_loss: 0.4199 - val_accuracy: 0.8408
Epoch 3/5
179/179 [==============================] - 1s 5ms/step - loss: 0.3823 - accuracy: 0.8545 - val_loss: 0.4285 - val_accuracy: 0.8327
Epoch 4/5
179/179 [==============================] - 1s 4ms/step - loss: 0.3822 - accuracy: 0.8541 - val_loss: 0.4704 - val_accuracy: 0.8412
Epoch 5/5
179/179 [==============================] - 1s 5ms/step - loss: 0.3638 - accuracy: 0.8579 - val_loss: 0.5498 - val_accuracy: 0.8412
CPU times: total: 26 s
Wall time: 4.9 s


In [3]:
%%time
# Обучениe медведя
y_train_cat_bear = keras.utils.to_categorical(bearops, 2)

model_bear = keras.Sequential([
    Flatten(input_shape=(236, 8, 1)),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')])

model_bear.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model_bear.fit(x_train, y_train_cat_bear, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
179/179 [==============================] - 1s 3ms/step - loss: 0.2834 - accuracy: 0.9187 - val_loss: 0.2678 - val_accuracy: 0.9302
Epoch 2/5
179/179 [==============================] - 0s 2ms/step - loss: 0.2559 - accuracy: 0.9217 - val_loss: 0.2688 - val_accuracy: 0.9302
Epoch 3/5
179/179 [==============================] - 0s 2ms/step - loss: 0.2434 - accuracy: 0.9246 - val_loss: 0.2873 - val_accuracy: 0.9292
Epoch 4/5
179/179 [==============================] - 0s 2ms/step - loss: 0.2385 - accuracy: 0.9249 - val_loss: 0.2884 - val_accuracy: 0.9271
Epoch 5/5
179/179 [==============================] - 0s 2ms/step - loss: 0.2313 - accuracy: 0.9278 - val_loss: 0.2947 - val_accuracy: 0.9302
CPU times: total: 7.48 s
Wall time: 2.3 s


In [6]:
%%time
# Предсказание на ближайший день
DF_curr = pd.DataFrame()
with Client(trtoken) as client:
    i = 1
    while DF_curr.shape[0] < 236:
        r = client.market_data.get_candles(
                figi="BBG004730ZJ9", # VTBR
                from_=datetime.utcnow() - timedelta(days=0) - timedelta(days=i),
                to=datetime.utcnow() - timedelta(days=0) - timedelta(days=i-1),
                interval=CandleInterval.CANDLE_INTERVAL_HOUR
            )
        df = create_df(r.candles)
        DF_curr = pd.concat([df, DF_curr], axis=0, ignore_index=True)
        i += 1

DF_lastline = DF_curr.tail(1)
DF_curr['dayofyear'] = DF_curr.time.dt.dayofyear
DF_curr['weekday'] = DF_curr.time.dt.weekday
DF_curr['utchour'] = DF_curr.time.dt.hour
DF_curr.drop(['time'], axis=1, inplace=True)

dfs_curr = DF_curr.tail(236).copy()

dfs_curr.volume = mmscaler(dfs_curr.volume)
dfs_curr.open = mmscaler(dfs_curr.open)
dfs_curr.close = mmscaler(dfs_curr.close)
dfs_curr.high = mmscaler(dfs_curr.high)
dfs_curr.low = mmscaler(dfs_curr.low)
dfs_curr.dayofyear = mmscaler(dfs_curr.dayofyear, ma=DF.dayofyear.max(), mi=DF.dayofyear.min())
dfs_curr.weekday = mmscaler(dfs_curr.weekday, ma=DF.weekday.max(), mi=DF.weekday.min())
dfs_curr.utchour = mmscaler(dfs_curr.utchour, ma=DF.utchour.max(), mi=DF.utchour.min())

x_current = list()
x_current.append(np.array(dfs_curr))
x_current = np.array(x_current)

t_bull = model_bull.predict(x_current, verbose=0)[0][1]
t_bear = model_bear.predict(x_current, verbose=0)[0][1]

tabl = pd.Series({'msktime': (DF_lastline.time+timedelta(hours=3)).values[0].astype('datetime64[s]'),
        'dayofyear': DF_lastline.time.dt.dayofyear.values[0],
        'today': (pd.Timestamp.utcnow()+timedelta(hours=3)).dayofyear,
        'volume': DF_lastline.volume.values[0],
        'open': DF_lastline.open.values[0],
        'close': DF_lastline.close.values[0],
        'high': DF_lastline.high.values[0],
        'low': DF_lastline.low.values[0],
        'growth by 3%': f'{t_bull:.2%}',
        'bulltarget': DF_lastline.close.values[0]*(1.03),
        'drop by 3%': f'{t_bear:.2%}',
        'beartarget': DF_lastline.close.values[0]*(0.97)})

print(tabl, '\n')

try:
    prediction_table = pd.read_excel(r'C:\SEG\myproj\trobot\prediction_VTBR.xlsx', index_col=0) # подставить свое
except:
    prediction_table = pd.DataFrame(columns=['msktime', 'dayofyear', 'today', 'volume', 'open', 'close', 'high',
                                             'low', 'growth by 3%', 'bulltarget', 'drop by 3%', 'beartarget'])

msktime         2023-03-12T18:00:00
dayofyear                        71
today                            71
volume                          827
open                       0.016525
close                      0.016525
high                        0.01653
low                         0.01639
growth by 3%                  1.62%
bulltarget                 0.017021
drop by 3%                    0.78%
beartarget                 0.016029
dtype: object 

CPU times: total: 109 ms
Wall time: 739 ms


In [5]:
prediction_table.loc[prediction_table.shape[0]] = list(tabl.values)
prediction_table.drop_duplicates(inplace=True)
prediction_table.to_excel(r'C:\SEG\myproj\trobot\prediction_VTBR.xlsx') # подставить свое